# 101: Hello world tutorial

From Machine Learning model to scalable deployment with Analitico.

_Read the full documentation in the [Documents](https://analitico.ai/app/recipes/rx_helloworld/markdown/README.md) page._

## Model
To keep things simple we use the iris dataset to train a SVM classifier.

In [1]:
import analitico
import pickle
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC

# Load sample dataset
iris = datasets.load_iris()

X = iris.data
y = iris.target

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Use SVM classifier 
model = SVC(kernel='linear').fit(X_train, y_train)

# Calculate test prediction
y_pred = model.predict(X_test)

# Save metrics for later training evaluation
analitico.set_model_metrics(y_test, y_pred, model, is_classifier=True, target_names=iris.target_names)

# Test the model accuracy
score = model.score(X_test,y_test.ravel())
print("Model accuracy: {:.2%}".format(score))

# Save the model for later predictions from endpoint
pickle.dump(model, open('iris_svm_model.pkl', 'wb'))

Model accuracy: 96.67%


## Analitico serverless handler

The serverless endpoint is set to call the `handle(event, **kwargs)` method with a set of data required by the model.
Here we load the model, execute the prediction with the input data and return the result.

In [2]:
# This cell is executed in a different context.
# All imports are declared here and all the dependecies it needs
# to run can be specified here or in the requirements.txt file

import json
import numpy as np
import pickle

# Retrieve the model here to be used between requests
model = pickle.load(open('iris_svm_model.pkl', "rb"))
    
def handle(event, **kwargs):
    """ Method called by the serverless endpoint url with the data for prediction """
  
    data = []
    data.append(list(event.values()))
    
    y_pred = model.predict(np.array(data))
    
    return {"prediction": int(y_pred[0]) }
  
def test(**kwargs):
    """ Test method can be called by Analitico to verify the model is working """
  
    data = {
      "sepal_length": 6.3,
      "sepal_width": 2.3,
      "petal_length": 4.4,
      "petal_width": 1.3
    }

    results = handle(data)
    print(json.dumps(results, indent=2))
  
    # we expect our prediction to be the iris Versicolour (class 1)
    print("\nTest passed: ")
    return results["prediction"] == 1

## Test the endpoint

Finally, let's test the endpoint with sample data.

In [3]:
test()

{
  "prediction": 1
}

Test passed: 


True